# Week5 - Logistic Regression Assignment

- Run the below cells.
- Complete all of the numbered questions. You may call any packages that we've used in class.  

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/classification/loan_status_data/loan_status.csv')

## Data Definitions

<ul>
<li><strong>Loan_ID</strong>: A unique loan ID.</li>
<li><strong>Gender</strong>: Either male or female.</li>
<li><strong>Married</strong>: Weather Married(yes) or Not Marttied(No).</li>
<li><strong>Dependents</strong>: Number of persons depending on the client.</li>
<li><strong>Education</strong>: Applicant Education(Graduate or Undergraduate).</li>
<li><strong>Self_Employed</strong>: Self-employed (Yes/No).</li>
<li><strong>ApplicantIncome</strong>: Applicant income.</li>
<li><strong>CoapplicantIncome</strong>: Co-applicant income.</li>
<li><strong>LoanAmount</strong>: Loan amount in thousands.</li>
<li><strong>Loan_Amount_Term</strong>: Terms of the loan in months.</li>
<li><strong>Credit_History</strong>: Credit history meets guidelines.</li>
<li><strong>Property_Area</strong>: Applicants are living either Urban, Semi-Urban or Rural.</li>
<li><strong>Loan_Status</strong>: Loan approved (Y/N).</li>
</ul>

In [10]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
1,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            381 non-null    object 
 1   Gender             376 non-null    object 
 2   Married            381 non-null    object 
 3   Dependents         373 non-null    object 
 4   Education          381 non-null    object 
 5   Self_Employed      360 non-null    object 
 6   ApplicantIncome    381 non-null    int64  
 7   CoapplicantIncome  381 non-null    float64
 8   LoanAmount         381 non-null    float64
 9   Loan_Amount_Term   370 non-null    float64
 10  Credit_History     351 non-null    float64
 11  Property_Area      381 non-null    object 
 12  Loan_Status        381 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 38.8+ KB


In [12]:
df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,381.000000,381.000000,381.000000,370.000000,351.000000
mean,3579.845144,1277.275381,104.986877,340.864865,0.837607
std,1419.813818,2340.818114,28.358464,68.549257,0.369338
min,150.000000,0.000000,9.000000,12.000000,0.000000
25%,2600.000000,0.000000,90.000000,360.000000,1.000000
50%,3333.000000,983.000000,110.000000,360.000000,1.000000
75%,4288.000000,2016.000000,127.000000,360.000000,1.000000
max,9703.000000,33837.000000,150.000000,480.000000,1.000000


# Question 1
- Any data quality issues/concerns? 
- Any clean up needed?

we can check if there are null values in columns and fill null with median values

In [13]:
df.isnull().sum()

Loan_ID               0
Gender                5
Married               0
Dependents            8
Education             0
Self_Employed        21
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term     11
Credit_History       30
Property_Area         0
Loan_Status           0
dtype: int64

In [14]:
median=df['Loan_Amount_Term'].median()
df['Loan_Amount_Term'].fillna(median, inplace=True)

In [15]:
median=df['Credit_History'].median()
df['Credit_History'].fillna(median, inplace=True)

In [17]:
df['Gender'].fillna('Male', inplace=True)
df['Dependents'].fillna('0', inplace=True)
df['Self_Employed'].fillna('No', inplace=True)

In [18]:
df.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

# Question 2
- What is the distribution of the target (`Loan_Status`)?  
- Do you have any concerns on class imbalances?

In [20]:
loan_status_distribution=df['Loan_Status'].value_counts()
print(loan_status_distribution)

Loan_Status
Y    271
N    110
Name: count, dtype: int64


class imbalance can lead the model to be biased towards majority class yes(Y) leading to poor accuracy while predicting minority class No(N)

# Question 3
- Create and print a list of the variables that you would treat as numerical and another list for the variables that you would treat as categorical.  
- Explain your choices.

In [32]:
numerical_vars = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']
categorical_vars = ['Loan_ID','Gender','Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']

I considered Loan_ID, Gender, Married, Dependents, Eduction, Self_Employed, Property_Area as categorical variables becuase they represent different categories or groups. Like, gender has value male and female

# Question 4
- Split the data into training and test sets.  
- Use 20% of the data for test and a random state of 120.  

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score
import seaborn as sns
from sklearn.linear_model import LogisticRegression

X=df.drop('Loan_Status', axis=1)
y=df['Loan_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=120)

print('Training set shape', X_train.shape, y_train.shape)
print('Test set shape', X_test.shape, y_test.shape)

Training set shape (304, 12) (304,)
Test set shape (77, 12) (77,)


# Question 5
- Create a pipeline to process the numerical data.  
- Create a pipeline to process the categorical data.  

Verify each pipeline contains the columns you would expect using a fit_transform on the training data, i.e., print the shapes of the fit_transforms for each pipeline.

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer

numerical_transformer=Pipeline([('imputer', SimpleImputer(strategy='median')),
                               ('scaler', StandardScaler())
                               ])

categorical_transformer=Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                                  ('onehot', OneHotEncoder(handle_unknown='ignore'))
                                 ])

In [43]:
X_train_numerical=X_train[numerical_vars]
numerical_transformed=numerical_transformer.fit_transform(X_train_numerical)
print('numerical pipeline transformed shape', numerical_transformed.shape)

X_train_categorical=X_train[categorical_vars]
categorical_transformed=categorical_transformer.fit_transform(X_train_categorical)
print('categorical pipeline transfromed shape', categorical_transformed.shape)

numerical pipeline transformed shape (304, 5)
categorical pipeline transfromed shape (304, 319)


# Question 6
- Create a pipeline that combines the pre-processing and implements a logistic regression model.  
- Print the accuracy on the training set and the test set.
- Do you have any concerns of overfitting based on the differences between the two accuracy scores?

In [35]:
processing_pipeline=ColumnTransformer(transformers=[('numerical', numerical_transformer, numerical_vars),
                                                    ('categorical', categorical_transformer, categorical_vars)])
print(processing_pipeline)

modeling_pipeline = Pipeline([
        ('data_processing', processing_pipeline),
        ('logreg', LogisticRegression(penalty=None))]
    )
modeling_pipeline

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['ApplicantIncome', 'CoapplicantIncome',
                                  'LoanAmount', 'Loan_Amount_Term',
                                  'Credit_History']),
                                ('categorical',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Loan_ID', 'Gender', 'Married', 'Dependents',
                

Pipeline(steps=[('data_processing',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['ApplicantIncome',
                                                   'CoapplicantIncome',
                                                   'LoanAmount',
                                                   'Loan_Amount_Term',
                                                   'Credit_History']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Loan_ID', 'Gender',
                                                   'Married', 'Dependents',
                                                   'Education', 'Self_Employed',
                                                   'Property_Area'])])),
                ('logreg', LogisticRegression(penalty=None))])

In [37]:
from sklearn.metrics import accuracy_score

modeling_pipeline.fit(X_train,y_train)

y_train_pred=modeling_pipeline.predict(X_train)
y_test_pred=modeling_pipeline.predict(X_test)

train_accuracy=accuracy_score(y_train,y_train_pred)
test_accuracy=accuracy_score(y_test,y_test_pred)

print('Accuracy on training set', train_accuracy)
print('Accuracy on test set', test_accuracy)

Accuracy on training set 1.0
Accuracy on test set 0.8701298701298701


yes, there seems to be an overfitting issue because the accuracy on training set(1.0) is substantially higher than accuracy on test set(0.87). It suggests that this high accuracy score can be because the model may have memorized the training data, leading to reduced performance on test set.

# Question 7
What would you recommend as potential next steps for continuing to develop and evaluate a model?

To address overfitting, we can use techniques like regularization, cross validation or also try to use more training data to help mitigate overfitting.